<a href="https://colab.research.google.com/github/BruceXavierChou/stock_analysis/blob/main/%E3%80%8Cstk_ch06_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH-06 個股分析機器人

### 1️⃣ 安裝及匯入套件

In [ ]:
!pip install openai
!pip install yfinance
from openai import OpenAI, OpenAIError # 串接 OpenAI API
import yfinance as yf
import pandas as pd # 資料處理套件
import numpy as np
import datetime as dt # 時間套件
import requests
from bs4 import BeautifulSoup

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


### 2️⃣ 輸入 OpenAI API KEY

In [ ]:
import getpass # 保密輸入套件
api_key = getpass.getpass("請輸入金鑰：")
client = OpenAI(api_key = api_key) # 建立 OpenAI 物件

請輸入金鑰：··········


### 3️⃣ 取得股價資料

In [ ]:
# 從 yfinance 取得一周股價資料
def stock_price(stock_id="大盤", days = 10):
  if stock_id == "大盤":
    stock_id="^TWII"
  else:
    stock_id += ".TW"

  end = dt.date.today() # 資料結束時間
  start = end - dt.timedelta(days=days) # 資料開始時間
  # 下載資料
  df = yf.download(stock_id, start=start)

  # 更換列名
  df.columns = ['開盤價', '最高價', '最低價',
                '收盤價', '調整後收盤價', '成交量']

  data = {
    '日期': df.index.strftime('%Y-%m-%d').tolist(),
    '收盤價': df['收盤價'].tolist(),
    '每日報酬': df['收盤價'].pct_change().tolist(),
    '漲跌價差': df['調整後收盤價'].diff().tolist()
    }

  return data

print(stock_price("2330"))

[*********************100%%**********************]  1 of 1 completed

{'日期': ['2024-07-08', '2024-07-09', '2024-07-10', '2024-07-11', '2024-07-12', '2024-07-15', '2024-07-16', '2024-07-17', '2024-07-18'], '收盤價': [1035.0, 1040.0, 1045.0, 1080.0, 1040.0, 1040.0, 1055.0, 1030.0, 1005.0], '每日報酬': [nan, 0.004830917874396157, 0.004807692307692291, 0.03349282296650724, -0.03703703703703709, 0.0, 0.014423076923076872, -0.023696682464455, -0.024271844660194164], '漲跌價差': [nan, 5.0, 5.0, 35.0, -40.0, 0.0, 15.0, -25.0, -25.0]}


### 4️⃣ 取得基本面資料

In [ ]:
# 基本面資料
def stock_fundamental(stock_id= "大盤"):
  if stock_id == "大盤":
      return None

  stock_id += ".TW"
  stock = yf.Ticker(stock_id)

  # 營收成長率
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)

  # 每季EPS
  quarterly_eps = np.round(stock.quarterly_financials.loc["Basic EPS"].dropna().tolist(), 2)

  # EPS季增率
  quarterly_eps_growth = np.round(stock.quarterly_financials.loc["Basic EPS"].pct_change(-1).dropna().tolist(), 2)

  # 轉換日期
  dates = [date.strftime('%Y-%m-%d') for date in stock.quarterly_financials.columns]

  data = {
      '季日期': dates[:len(quarterly_revenue_growth)],
      '營收成長率': quarterly_revenue_growth.tolist(),
      'EPS': quarterly_eps[0:3].tolist(),
      'EPS 季增率': quarterly_eps_growth[0:3].tolist()
  }

  return data

print(stock_fundamental("2330"))

{'季日期': ['2024-03-31', '2023-12-31', '2023-09-30', '2023-06-30', '2023-03-31'], '營收成長率': [-0.05, 0.14, 0.14, -0.05, 0.0], 'EPS': [8.7, 9.21, 8.14], 'EPS 季增率': [-0.06, 0.13, 0.16]}


### 5️⃣ 取得新聞資料

In [ ]:
# 新聞資料
def stock_news(stock_name ="大盤"):
  if stock_name == "大盤":
    stock_name="台股 -盤中速報"

  data=[]
  # 取得 Json 格式資料
  json_data = requests.get(f'https://ess.api.cnyes.com/ess/api/v1/news/keyword?q={stock_name}&limit=5&page=1').json()

  # 依照格式擷取資料
  items=json_data['data']['items']
  for item in items:
      # 網址、標題和日期
      news_id = item["newsId"]
      title = item["title"]
      publish_at = item["publishAt"]
      # 使用 UTC 時間格式
      utc_time = dt.datetime.utcfromtimestamp(publish_at)
      formatted_date = utc_time.strftime('%Y-%m-%d')
      # 前往網址擷取內容
      url = requests.get(f'https://news.cnyes.com/'
                        f'news/id/{news_id}').content
      soup = BeautifulSoup(url, 'html.parser')
      p_elements=soup .find_all('p')
      # 提取段落内容
      p=''
      for paragraph in p_elements[4:]:
          p+=paragraph.get_text()
      data.append([stock_name, formatted_date ,title,p])
  return data

print(stock_news("台積電"))

[['台積電', '2024-07-18', '被AI顛覆的5張半年報', '也由於幾乎所有有意搶攻ＡＩ龐大商機的國際巨擘均正與台積電合作，因此台積電先前也預期伺服器ＡＩ處理器今年營收獻將成長逾一倍、占比達十一至十三％，未來五年的營收年複合成長率（ＣＡＧＲ）則將達五○％，至二○二八年營收占比估超過兩成。市場也預期台積電今年ＥＰＳ可達四○元，明、後年則上看五○、六○元，有望逐年成長。而台積電股價今年來也是一路狂飆，今年迄今漲幅最高達八二％，並突破千元大關，最高寫下一○八○元的歷史新高水準，對比整體加權指數增長不到四成。且目前ＡＤＲ換算台股價格也來到一二○○元，或有助台積電後市表現。除台積電外，今年不少ＡＩ概念股仍持續受市場關注，從公告六月營收來看，整體上市櫃公司合併營收已連四個月突破三兆大關，六月營收以三．六四兆元締造同期次高，月增○．四一％、年增十一．三八％，僅次於二二年的三．八六兆元。總計第二季營收達十．七七兆元，季增九．八二％、年增十四．七三％；帶動上半年合併營收突破二○兆大關，達二○．五七兆元，年增九．九八％。進一步來看，上市櫃公司六月營收創新高家數達六三家，其中又以ＡＩ概念股表現強勁，像是散熱雙雄雙鴻、奇鋐，ASIC 指標的世芯ＫＹ，機殼廠晟銘電，ＣＣＬ的台光電等均入列。另外，台積電 CoWoS 先進封裝產能吃緊，帶動相關廠商如旺矽、萬潤、辛耘六月營收亦皆同步改寫新高。由此，生成式ＡＩ的蓬勃發展，對台灣企業來說，絕對是搭上一大營運成長列車，尤其，隨著在雲端服務大廠（ＣＳＰ）積極搶奪ＡＩ市占率的同時，也讓客製化晶片需求大幅增加，助攻台灣相關的ＩＣ設計服務業者及矽智財（ＩＰ）業者接單表現，包括世芯ＫＹ、創意、智原，甚至是正在布局客製化處理器晶片市場的聯發科、聯詠、瑞昱都有望受惠。（全文未完）來源：《先探投資週刊》2309 期更多精彩內容請至 《先探投資週刊》下一篇'], ['台積電', '2024-07-18', '9月降息股民教戰守則', '回顧川普二○一六年當選總統，市場對於減稅、放鬆金融管制與基礎建設等政策反應佳，尤其是傳統石化、加密貨幣等產業；此外，川普致力於讓美國再次偉大，保護主義下的汽車、鋼鐵乃至半導體，也都有機會受惠。而美中貿易戰的延伸也相當受到矚目，拜登與川普對中國態度，基本上都相當強硬，然而川普恐更為激進，若重返白宮兌現對中國商品開徵六○％關稅

### 6️⃣ 爬取股號、股名對照表

In [ ]:
# 取得全部股票的股號、股名
def stock_name():
  print("線上讀取股號、股名、及產業別")

  response = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
  url_data = BeautifulSoup(response.text, 'html.parser')
  stock_company = url_data.find_all('tr')

  # 資料處理
  data = [
      (row.find_all('td')[0].text.split('\u3000')[0].strip(),
        row.find_all('td')[0].text.split('\u3000')[1],
        row.find_all('td')[4].text.strip())
      for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
  ]

  df = pd.DataFrame(data, columns=['股號', '股名', '產業別'])

  return df

name_df = stock_name()

線上讀取股號、股名、及產業別


### 7️⃣ 取得股票名稱

In [ ]:
# 取得股票名稱
def get_stock_name(stock_id, name_df):
    return name_df.set_index('股號').loc[stock_id, '股名']

print(name_df.head())
print("--------------------------")
print(get_stock_name("1417",name_df))

     股號  股名   產業別
0  1101  台泥  水泥工業
1  1102  亞泥  水泥工業
2  1103  嘉泥  水泥工業
3  1104  環泥  水泥工業
4  1108  幸福  水泥工業
--------------------------
嘉裕


### 8️⃣ 建構 GPT 3.5 模型

In [ ]:
# 建立 GPT 3.5-16k 模型
def get_reply(messages):
    try:
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages = messages
        )
        reply = response.choices[0].message.content
    except OpenAIError as err:
        reply = f"發生 {err.type} 錯誤\n{err.message}"
    return reply

# 建立訊息指令(Prompt)
def generate_content_msg(stock_id, name_df):

    stock_name = get_stock_name(
        stock_id, name_df) if stock_id != "大盤" else stock_id

    price_data = stock_price(stock_id)
    news_data = stock_news(stock_name)

    content_msg = f'請依據以下資料來進行分析並給出一份完整的分析報告:\n'

    content_msg += f'近期價格資訊:\n {price_data}\n'

    if stock_id != "大盤":
        stock_value_data = stock_fundamental(stock_id)
        content_msg += f'每季營收資訊：\n {stock_value_data}\n'

    content_msg += f'近期新聞資訊: \n {news_data}\n'
    content_msg += f'請給我{stock_name}近期的趨勢報告,請以詳細、\
      嚴謹及專業的角度撰寫此報告,並提及重要的數字, reply in 繁體中文'

    return content_msg

# StockGPT
def stock_gpt(stock_id, name_df=name_df):
    content_msg = generate_content_msg(stock_id, name_df)

    msg = [{
        "role": "system",
        "content": f"你現在是一位專業的證券分析師, 你會統整近期的股價\
      、基本面、新聞資訊等方面並進行分析, 然後生成一份專業的趨勢分析報告"
    }, {
        "role": "user",
        "content": content_msg
    }]

    reply_data = get_reply(msg)

    return reply_data


### 9️⃣ 大盤趨勢報告

In [ ]:
reply = stock_gpt(stock_id="大盤")
print(reply)

[*********************100%%**********************]  1 of 1 completed


**股市趨勢分析報告**

**1. 近期股市走勢分析:**
根據最近價格資訊顯示，股市走勢較為不穩定。7月18日收盤指數為23398.47，較前一個交易日下跌1.56%，漲跌幅度較為明顯。此外，從收盤價走勢來看，股市在過去幾日呈現震蕩下跌的趨勢，表現較為疲軟。

**2. 新聞對股市影響分析:**
近期新聞資訊中提到了台積電法說會的影響，這對股市造成了一定的衝擊。台積電在法說會中降低了半導體市場展望，這導致投資人情緒較為保守，也造成了台積電及整體股市的走跌。另外，美國前總統特朗普的相對不利談話以及艾斯摩爾財報表現不佳也對市場造成了一定的影響。

**3. 技術面分析:**
從技術面來看，股市在過去幾日呈現下跌趨勢，收盤價跌破月均線支撐，並出現跳空重挫的情況。短線均線也出現交叉下彎的現象，顯示市場走勢較為疲軟。同時，指數在回檔過程中，KD指標也轉為弱勢，整體市場呈現整理的格局。

**4. 股市展望與建議:**
考慮到美股大跌、台積電重挫等因素，加上技術面呈現走弱的趨勢，以及全球地緣政治風險等因素對市場的影響，未來台股指數可能持續維持震盪整理的格局。投資人應保持謹慎態度，密切關注市場走勢變化，建議控制風險，適時調整投資組合。

**5. 重要數字摘要:**
- 7月18日收盤價: 23398.47
- 前一交易日收盤價: 23769.82
- 7月18日漲跌幅: -1.56%
- 最大跌點日期: 4月19日，單日跌幅3.81%，指數跌破月均線支撐

以上為近期股市走勢的分析報告，希望能對您的投資決策提供幫助。如需更多專業建議，建議諮詢財務專業人士。


### 🔟 個股分析報告

In [ ]:
reply = stock_gpt(stock_id="2330")
print(reply)

[*********************100%%**********************]  1 of 1 completed


### 台積電近期趨勢分析報告

#### 股價表現分析：
從近期的收盤價資訊觀察，台積電股價出現波動，7月11日一度達到1080元的歷史新高，但在接下來的幾天內有所回調，於7月18日收盤價來到1005元。收盤價波動主要受市場消息和整體股市走勢的影響。每日報酬率呈現波動走勢，顯示投資人在此段期間的交易活躍。

#### 基本面分析：
根據營收資訊，台積電在上半年取得不錯的業績表現，營收年增率達28%，並在第二季度法說會中指出，5奈米以下製程貢獻了50%以上的營收。另外，每股盈餘（EPS）在這段期間也達到了9.56元，創下同期新高，顯示公司營運持續穩定增長。

#### 新聞資訊影響分析：
近期新聞資訊反映了市場對於台積電未來發展的樂觀預期，特別是AI領域的商機，以及台積電在新技術領域的積極布局和未來發展潛力。然而，美國前總統川普相關的政策和全球經濟情勢等因素也會對股價產生一定的影響。

#### 技術面分析：
從技術面來看，台積電股價出現與年線明顯乖離的情況，且相關技術指標如KD值也呈現反轉的現象，顯示股價可能處於高位，成為回檔修正的風險因素。投資者應留意股價與技術指標之間的關聯，以及追蹤股價與均線的走勢。

#### 結論：
綜合以上分析，台積電作為一家龍頭半導體公司，在營收和EPS方面呈現穩健增長，市場對其未來發展充滿信心。然而，股價波動和技術面的情況需要投資者關注，以避免盲目操作。投資者應注意新聞資訊對市場的影響，同時保持警惕，根據基本面和技術面因素做出理性的投資決策。

以上分析僅供參考，投資有風險，建議投資者在了解風險的前提下做出合理的投資決策。


### 1️⃣1️⃣ 雞蛋水餃股也能做分析

In [ ]:
reply = stock_gpt(stock_id="4414")
print(reply)

[*********************100%%**********************]  1 of 1 completed


### 如興（代號：4414）近期趨勢分析報告

#### 股價資訊分析
- **收盤價趨勢：** 近期股價波動幅度較大，自2024年7月8日至7月18日的交易日中，股價從2.91元上升至3.88元，漲幅達到約0.97元，整體呈現上升趨勢。
- **每日報酬率：** 從每日報酬率的變化來看，公司股價波動較大，報酬率存在較大的波動性，值得關注。
- **漲跌價差：** 在近期的交易中，公司股價的漲跌價差也較大，顯示市場對於該股存在一定程度的不確定性。

#### 基本面分析
- **營收成長率：** 2023年第一季度至2024年第一季度，營收成長率呈現起伏，整體營收成長幅度不穩定，需要持續關注。
- **EPS季增率：** 近期EPS季增率變動較大，波動性較高，需警惕其對公司業績影響。
- **公司財務狀況：** 根據營收及EPS表現，公司財務狀況存有一定風險，投資人應謹慎評估與判斷。

#### 新聞資訊分析
- **盈利及財務改善消息：** 有報導指出公司營收及獲利有顯著增加，但需注意消息真實性與對股價影響。
- **公司營運動態：** 從相關新聞中可以看出公司具有多元發展機會，如AI晶片市場布局及全球雲端服務供應商資本支出等，有望帶動公司未來業績增長。
- **市場情緒與動能指標：** 市場對如興股價存在一定的興奮情緒，動能指標呈現活潑的交易狀態，投資人應注意市場波動風險。

#### 結論與建議
- **趨勢綜合分析：** 綜合各項指標看，如興的股價、基本面及市場消息均具有波動性，投資風險相對較高，需謹慎評估。
- **投資建議：** 短期投資者可適時把握波段操作機會，長期投資者應持續關注公司營運動態與財務報表，緊盯市場風險及機會。

以上報告僅供參考，投資者在做出投資決策前應謹慎評估風險與盈利潛力，並考慮專業投資建議。
